# Differentiable Beamforming for Ultrasound Autofocusing


In this tutorial we will implement a basic [differential beamformer](https://waltersimson.com/dbua/). A gradient descent method to minimize a pixelwise common midpoint phase error. The algorithm is slightly simplified, loss is computed without patching. 

For more information we would like to refer you to the research [webpage](https://waltersimson.com/dbua/). 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tue-bmd/zea/blob/main/docs/source/notebooks/pipeline/dbua.ipynb)
&nbsp;
[![View on GitHub](https://img.shields.io/badge/GitHub-View%20Source-blue?logo=github)](https://github.com/tue-bmd/zea/blob/main/docs/source/notebooks/pipeline/dbua.ipynb)
&nbsp;
[![Hugging Face dataset](https://img.shields.io/badge/Hugging%20Face-Dataset-yellow?logo=huggingface)](https://huggingface.co/datasets/zeahub/circular_inclusion_simualation)

‼️ **Important:** This notebook is optimized for **GPU/TPU**. Code execution on a **CPU** may be very slow.

If you are running in Colab, please enable a hardware accelerator via:

**Runtime → Change runtime type → Hardware accelerator → GPU/TPU** 🚀.

In [1]:
%%capture
%pip install zea

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"
os.environ["ZEA_DISABLE_CACHE"] = "1"
os.environ["ZEA_LOG_LEVEL"] = "INFO"

In [3]:
import os
import zea
from zea import File, init_device
from zea.visualize import set_mpl_style
from zea.io_lib import matplotlib_figure_to_numpy, save_to_gif
import time
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt
import keras

device = init_device()
set_mpl_style()

zea: Using backend 'jax'


-------------------GPU settings-------------------
     memory
GPU        
0        5615
Selecting 1 GPU based on available memory.
Selected GPU 0 with Free Memory: 5615.00 MiB
--------------------------------------------------


# Load dataset 


Here we load a dataset, In this case a circular inclusion in an isoechoic medium. 
We set up a field of view and a pixelwise loss function. We also select a grid of points where we will be calculating the loss. This grid is more an indication of how many points we consider per  step as we resample them every iteration to prevent overfitting on specific locations.

In [4]:
file_path = "/workspace/circular_inclusion_simulation.hdf5"
file = File(file_path)
data_frame = file.load_data(data_type="raw_data")
probe = file.probe()
probe_geometry = probe.probe_geometry

xlims = (-15e-3, 15e-3)
x_min, x_max = xlims
zlims = (0e-3, 30e-3)
z_min, z_max = zlims
width, height = xlims[1] - xlims[0], zlims[1] - zlims[0]
wavelength = 1540 / probe.center_frequency

Nx = 25
Nz = 25
f_number = 0.7
scan = file.scan(xlims=xlims, zlims=zlims, grid_size_x=Nx, grid_size_z=Nz)

LossPipeline = zea.Pipeline(
    [
        zea.ops.PatchedGrid(
            [zea.ops.TOFCorrectionGrid(), zea.ops.CommonMidpointPhaseError()], num_patches=Nx * Nz
        ),
    ],
    jit_options="pipeline",
)
parameters = LossPipeline.prepare_parameters(probe, scan)
parameters["f_number"] = f_number

zea: WARNING No initial times provided, using zeros
zea: WARNING No azimuth angles provided, using zeros


# Set up SOS grid
here we define the grid of sound speed voxels that will be optimized.


In [5]:
x_sos_grid = np.linspace(x_min, x_max, 40)
z_sos_grid = np.linspace(z_min, z_max, 40)
initial_sound_speed = 1460
sos_grid = initial_sound_speed * np.ones((x_sos_grid.shape[0], z_sos_grid.shape[0]))

# Setup optimizer
Here we define the optimization schedule parameters.



In [6]:
def adam(step_size, b1=0.9, b2=0.999, eps=1e-8):
    """Construct optimizer triple for Adam.

    Implementation adapted from JAX's example: https://jax.readthedocs.io/en/latest/_modules/jax/example_libraries/optimizers.html#adam
    See example usage: https://jax.readthedocs.io/en/latest/jax.example_libraries.optimizers.html#jax.example_libraries.optimizers.adam

    Args:
    step_size: positive scalar
    b1: optional, a positive scalar value for beta_1, the exponential decay rate
        for the first moment estimates (default 0.9).
    b2: optional, a positive scalar value for beta_2, the exponential decay rate
        for the second moment estimates (default 0.999).
    eps: optional, a positive scalar value for epsilon, a small constant for
        numerical stability (default 1e-8).

    Returns:
    An (init_fun, update_fun, get_params) triple.
    """

    def init(x0):
        m0 = keras.ops.zeros_like(x0)
        v0 = keras.ops.zeros_like(x0)
        i = 0
        return x0, m0, v0, i

    def update(g, state):
        """Update rule for Adam optimizer.

        Args:
            g (array): gradient
            state (tuple): state of the optimizer
                (x, m, v, i) = (parameter, first moment estimate,
                second moment estimate, iteration count)

        Returns:
            state: updated state
        """
        x, m, v, i = state
        m = (1 - b1) * g + b1 * m  # First  moment estimate.
        v = (1 - b2) * keras.ops.square(g) + b2 * v  # Second moment estimate.
        mhat = m / (1 - b1 ** (i + 1))  # Bias correction.
        vhat = v / (1 - b2 ** (i + 1))
        x = x - step_size * mhat / (keras.ops.sqrt(vhat) + eps)
        i = i + 1
        return x, m, v, i

    def get_params(state):
        """Returns just the parameter from the state."""
        x, *_ = state
        return x

    return init, update, get_params


# === Initialize optimizer ===
num_iterations = 100
step_size = 1  # or whatever peak LR you were using
init_fn, update_fn, get_params_fn = adam(step_size)
opt_state = init_fn(sos_grid)  # sos_grid = your parameters

# Setup complete loss function
Here we combine the pixelwise loss with some vertical and horizontal regularizers. 
Furthermore, we introduce some helper functions for the optmization loop.

In [7]:
def loss_fn(sos_grid, x_sos_grid, z_sos_grid, LossPipeline, parameters, data_frame, flatgrid):
    dx_sos = x_sos_grid[1] - x_sos_grid[0]
    dz_sos = z_sos_grid[1] - z_sos_grid[0]
    parameters["flatgrid"] = flatgrid
    out = LossPipeline(
        data=data_frame,
        sos_grid=sos_grid,
        x_sos_grid=x_sos_grid,
        z_sos_grid=z_sos_grid,
        **parameters,
    )
    metric = out["data"]
    metric_safe = keras.ops.nan_to_num(metric, nan=0.0)
    metric_loss = keras.ops.mean(metric_safe)
    tvx = keras.ops.mean(keras.ops.square(keras.ops.diff(sos_grid, axis=0)))
    tvz = keras.ops.mean(keras.ops.square(keras.ops.diff(sos_grid, axis=1)))
    variation_loss = (tvx + tvz) * 1e2 * dx_sos * dz_sos
    total_loss = metric_loss + variation_loss
    return total_loss


loss_fn_caller = zea.backend.autograd.AutoGrad()
loss_fn_caller.set_function(loss_fn)


def compute_gradients(sos_grid, data_frame, flatgrid):
    kwargs = dict(
        x_sos_grid=x_sos_grid,
        z_sos_grid=z_sos_grid,
        LossPipeline=LossPipeline,
        parameters=parameters,
        data_frame=data_frame,
        flatgrid=flatgrid,
    )
    grad, loss = loss_fn_caller.gradient_and_value(sos_grid, **kwargs)

    return grad, loss


def apply_gradients(opt_state, grad):
    new_sos_grid, m, v, i = update_fn(grad, opt_state)
    new_opt_state = (new_sos_grid, m, v, i)
    return new_sos_grid, new_opt_state


def resample_grid(parameters, xlims, zlims):
    seed_generator = keras.random.SeedGenerator(int(time.time() * 1e6) % (2**32 - 1))
    n_pix = parameters["flatgrid"].shape[0]
    x = keras.random.uniform(
        shape=(n_pix,), minval=xlims[0] + 5e-3, maxval=xlims[1], seed=seed_generator
    )
    y = keras.ops.zeros_like(x)
    z = keras.random.uniform(shape=(n_pix,), minval=zlims[0], maxval=zlims[1], seed=seed_generator)

    coords = keras.ops.stack([x, y, z], axis=-1)
    return coords

# Make Visualisation Pipelines


In [8]:
Nxplot = int(width / (0.5 * wavelength) / 4) + 1
Nzplot = int(height / (0.5 * wavelength) / 4) + 1
gridplot = file.scan(xlims=xlims, zlims=zlims, grid_size_x=Nxplot, grid_size_z=Nzplot)
LossplotPipeline = zea.Pipeline(
    [
        zea.ops.PatchedGrid(
            [zea.ops.TOFCorrectionGrid(), zea.ops.CommonMidpointPhaseError()],
            num_patches=(Nzplot) * (Nxplot),
        ),
    ],
    jit_options="pipeline",
)
ImageplotPipeline = zea.Pipeline(
    [
        zea.ops.PatchedGrid(
            [zea.ops.TOFCorrectionGrid(), zea.ops.DelayAndSum()],
            num_patches=Nxplot * Nzplot,
        ),
        zea.ops.EnvelopeDetect(),
        zea.ops.Normalize(),
        zea.ops.LogCompress(),
    ],
    jit_options="pipeline",
)

parametersplot = LossplotPipeline.prepare_parameters(probe, gridplot)
parametersplot["demodulation_frequency"] = parameters["demodulation_frequency"]
parametersplot["f_number"] = f_number

zea: WARNING No initial times provided, using zeros
zea: WARNING No azimuth angles provided, using zeros


# Iterative optimization loop

Here we iteratively update the sound speed grid to minimze the common midpoint phase error.

In [9]:
viz_frames = []
loss_history = []

fig, (ax_bmode, ax_lossmap, ax_img, ax_loss) = plt.subplots(1, 4, figsize=(30, 4), dpi=100)


extent = [x_sos_grid.min(), x_sos_grid.max(), z_sos_grid.min(), z_sos_grid.max()]
bmodeim = ax_bmode.imshow(np.zeros((Nxplot, Nzplot)), extent=extent, cmap="gray", vmin=-60, vmax=0)
lossim = ax_lossmap.imshow(np.zeros((Nxplot, Nzplot)), extent=extent, cmap="gray", vmin=0, vmax=1)
im = ax_img.imshow(np.zeros((Nxplot, Nzplot)), extent=extent, cmap="jet", origin="lower")
im.set_clim(1440, 1500)
(loss_curve,) = ax_loss.plot([], [], "-o")


ax_bmode.set_title("Beamformed image")
ax_bmode.set_xlabel("x [mm]")
ax_bmode.set_ylabel("z [mm]")

ax_lossmap.set_title("CMPE Loss plot")
ax_lossmap.set_xlabel("x [mm]")
ax_lossmap.set_ylabel("z [mm]")

ax_img.set_title("Speed of Sound (SOS) Estimate")
ax_img.set_xlabel("x [mm]")
ax_img.set_ylabel("z [mm]")
ax_img.invert_yaxis()

ax_loss.set_title("CMPE mean Loss")
ax_loss.set_xlabel("Iteration")
ax_loss.set_ylabel("Loss")
ax_loss.set_xlim(0, num_iterations)
ax_loss.set_ylim(0.15, 0.2)

fig.colorbar(bmodeim, ax=ax_bmode, fraction=0.046, pad=0.04)
fig.colorbar(lossim, ax=ax_lossmap, fraction=0.046, pad=0.04)
fig.colorbar(im, ax=ax_img, fraction=0.046, pad=0.04)

for i in trange(num_iterations, desc="Epochs"):
    flatgrid = resample_grid(parameters, xlims, zlims)
    grad, loss = compute_gradients(sos_grid, data_frame, flatgrid)
    sos_grid, opt_state = apply_gradients(opt_state, grad)
    loss_history.append(float(loss))

    if i % 5 == 0:
        bmode = ImageplotPipeline(
            data=data_frame,
            sos_grid=sos_grid,
            x_sos_grid=x_sos_grid,
            z_sos_grid=z_sos_grid,
            **parametersplot,
        )["data"][0].reshape(Nxplot, Nzplot)

        lossimage = LossplotPipeline(
            data=data_frame,
            sos_grid=sos_grid,
            x_sos_grid=x_sos_grid,
            z_sos_grid=z_sos_grid,
            **parametersplot,
        )["data"][0].reshape(Nxplot, Nzplot)

        bmodeim.set_data(bmode)
        lossim.set_data(lossimage)
        im.set_data(np.array(sos_grid).T)

        loss_curve.set_data(np.arange(len(loss_history)), loss_history)

        # ax_loss.relim()
        # ax_loss.autoscale_view()
        viz_frames.append(matplotlib_figure_to_numpy(fig))

plt.close(fig)
save_to_gif(viz_frames, "sos_optimization.gif", shared_color_palette=True, fps=10)

zea: Successfully saved GIF to -> sos_optimization.gif
